In [1]:
# =========================================
# 📘 画像ファイル名一括変更スクリプト
# =========================================
# 🔹目的：
# images/information と images/Usage にある画像ファイルの名前を
# 「information_◯」「Usage_◯」のように番号付きでリネームします。

import os
import glob

# =========================================
# 📁 対象フォルダのパスを設定
# =========================================
base_dir = "images"  # ルートディレクトリ
folders = ["information", "Usage"]  # 対象サブフォルダ

# =========================================
# 📂 各フォルダ内のファイルを順に処理
# =========================================
for folder in folders:
    target_path = os.path.join(base_dir, folder)
    
    # 🔹指定フォルダ内の画像ファイルを取得（jpg, png, jpeg, bmp, gif対応）
    image_files = glob.glob(os.path.join(target_path, "*.*"))
    image_files = [f for f in image_files if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".gif"))]
    
    # 🔹1枚ずつリネーム
    for idx, file_path in enumerate(sorted(image_files), start=1):
        # 拡張子を取得
        ext = os.path.splitext(file_path)[1]
        
        # 新しいファイル名を作成
        new_name = f"{folder}_{idx}{ext}"
        new_path = os.path.join(target_path, new_name)
        
        # 🔹ファイル名を変更
        os.rename(file_path, new_path)
        print(f"✅ {file_path} → {new_path}")

print("\n🎉 すべてのファイル名変更が完了しました！")


✅ images\information\information_1.jpeg → images\information\information_1.jpeg
✅ images\information\information_2.jpeg → images\information\information_2.jpeg
✅ images\information\information_3.jpeg → images\information\information_3.jpeg
✅ images\Usage\Usage_1.jpeg → images\Usage\Usage_1.jpeg
✅ images\Usage\Usage_2.jpeg → images\Usage\Usage_2.jpeg
✅ images\Usage\Usage_3.jpeg → images\Usage\Usage_3.jpeg

🎉 すべてのファイル名変更が完了しました！


In [2]:
# ============================================================
# 📘 画像一括処理：表検出・分割・cut_rows保存対応版
# ============================================================


import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
from datetime import datetime

# =========================================
# 📁 入出力ディレクトリ設定
# =========================================
input_root = Path("images")
folders = ["information", "Usage"]

# export/process/日付_時刻 のフォルダを作成
today_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
export_root = Path("export") / "process" / today_str
export_root.mkdir(parents=True, exist_ok=True)

# 切り抜き画像をまとめるフォルダ
cut_rows_dir = Path("cut_rows")  # ← ここを Path にする！
cut_rows_dir.mkdir(parents=True, exist_ok=True)


# =========================================
# 📦 各フォルダ内の画像を順に処理
# =========================================
for folder in folders:
    input_dir = input_root / folder
    image_files = [f for f in input_dir.glob("*.*") if f.suffix.lower() in [".jpg", ".jpeg", ".png", ".bmp", ".gif"]]

    print(f"\n=== 📂 {folder} フォルダ: {len(image_files)} 枚の画像を処理します ===")

    for idx, img_path in enumerate(sorted(image_files), start=1):
        print(f"\n▶ 処理中: {img_path.name}")

        # --- 出力先ディレクトリを作成 ---
        output_dir = export_root / f"{folder}_{idx}"
        output_dir.mkdir(parents=True, exist_ok=True)

        # ===============================
        # Step 1: 画像の読み込み
        # ===============================
        img = cv2.imread(str(img_path))
        if img is None:
            print("⚠ 読み込み失敗:", img_path)
            continue

        # ===============================
        # Step 2: グレースケール変換
        # ===============================
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(str(output_dir / f"{folder}_gray.jpg"), gray)

        # ===============================
        # Step 3: 二値化
        # ===============================
        binary = cv2.adaptiveThreshold(
            gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 15, 10
        )
        cv2.imwrite(str(output_dir / f"{folder}_binary.jpg"), binary)

        # ===============================
        # Step 4: モルフォロジー処理
        # ===============================
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
        morphed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=2)
        cv2.imwrite(str(output_dir / f"{folder}_morphed.jpg"), morphed)

        # ===============================
        # Step 5: 輪郭抽出
        # ===============================
        contours, _ = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        img_contours = img.copy()
        cv2.drawContours(img_contours, contours, -1, (0, 255, 0), 1)
        cv2.imwrite(str(output_dir / f"{folder}_contours.jpg"), img_contours)

        # ===============================
        # Step 6: 表領域の抽出
        # ===============================
        table_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 10000]
        if not table_contours:
            print("⚠ 表領域が検出されませんでした。")
            continue

        table_cnt = max(table_contours, key=cv2.contourArea)
        peri = cv2.arcLength(table_cnt, True)
        approx = cv2.approxPolyDP(table_cnt, 0.02 * peri, True)

        # ===============================
        # Step 7: 透視変換による補正
        # ===============================
        if len(approx) == 4:
            pts = approx.reshape(4, 2)

            def order_points(pts):
                rect = np.zeros((4, 2), dtype="float32")
                s = pts.sum(axis=1)
                rect[0] = pts[np.argmin(s)]
                rect[2] = pts[np.argmax(s)]
                diff = np.diff(pts, axis=1)
                rect[1] = pts[np.argmin(diff)]
                rect[3] = pts[np.argmax(diff)]
                return rect

            rect = order_points(pts)
            (tl, tr, br, bl) = rect
            widthA = np.linalg.norm(br - bl)
            widthB = np.linalg.norm(tr - tl)
            heightA = np.linalg.norm(tr - br)
            heightB = np.linalg.norm(tl - bl)
            maxWidth = int(max(widthA, widthB))
            maxHeight = int(max(heightA, heightB))

            dst = np.array([
                [0, 0],
                [maxWidth - 1, 0],
                [maxWidth - 1, maxHeight - 1],
                [0, maxHeight - 1]
            ], dtype="float32")

            M = cv2.getPerspectiveTransform(rect, dst)
            table_img = cv2.warpPerspective(img, M, (maxWidth, maxHeight))
        else:
            x, y, w, h = cv2.boundingRect(table_cnt)
            table_img = img[y:y+h, x:x+w]

        cv2.imwrite(str(output_dir / f"{folder}_table.jpg"), table_img)

        # ===============================
        # Step 8: 横線検出と分割
        # ===============================
        gray_table = cv2.cvtColor(table_img, cv2.COLOR_BGR2GRAY)
        thresh_table = cv2.adaptiveThreshold(
            gray_table, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 15, 10
        )
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
        opened = cv2.morphologyEx(thresh_table, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)

        contours, _ = cv2.findContours(opened, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[1])
        y_positions = [cv2.boundingRect(c)[1] for c in contours]

        if len(y_positions) < 2:
            print("⚠ 横線が検出されませんでした。")
            continue

        # --- 15行ごとに分割 ---
        for cut_idx, i in enumerate(range(0, len(y_positions) - 1, 15), start=1):
            y1 = y_positions[i]
            y2 = y_positions[min(i + 15, len(y_positions) - 1)]
            row_img = table_img[y1:y2, :]

            # 例: information_1_1.jpg
            save_name = f"{folder}_{idx}_{cut_idx}.jpg"
            save_path = cut_rows_dir / save_name
            cv2.imwrite(str(save_path), row_img)

        print(f"✅ {img_path.name} → 処理完了。切り抜き保存先: {cut_rows_dir}")

print("\n🎉 全画像の処理が完了しました！")



=== 📂 information フォルダ: 3 枚の画像を処理します ===

▶ 処理中: information_1.jpeg
✅ information_1.jpeg → 処理完了。切り抜き保存先: cut_rows

▶ 処理中: information_2.jpeg
✅ information_2.jpeg → 処理完了。切り抜き保存先: cut_rows

▶ 処理中: information_3.jpeg
✅ information_3.jpeg → 処理完了。切り抜き保存先: cut_rows

=== 📂 Usage フォルダ: 3 枚の画像を処理します ===

▶ 処理中: Usage_1.jpeg
✅ Usage_1.jpeg → 処理完了。切り抜き保存先: cut_rows

▶ 処理中: Usage_2.jpeg
✅ Usage_2.jpeg → 処理完了。切り抜き保存先: cut_rows

▶ 処理中: Usage_3.jpeg
✅ Usage_3.jpeg → 処理完了。切り抜き保存先: cut_rows

🎉 全画像の処理が完了しました！


In [3]:
# ===============================
# 📘 Step 1: GPT-4 Vision APIで表画像の文字起こし（複数画像対応・コスト算出付き）
# ===============================
import os
from pathlib import Path
from openai import OpenAI
import base64
import pandas as pd

# --- 1️⃣ APIクライアントを初期化 ---
client = OpenAI()

# --- 2️⃣ OCR対象フォルダを指定 ---
folder_path = "cut_rows"

# --- 3️⃣ OCRプロンプト設定 ---
prompt_text = """
PDFもしくは画像データを文字起こしします。
文字起こしするデータは表形式で、列は以下の6列です。
カセットNo. / 薬品コード / 薬品名称 / 現在量 / 警告量 / 前回登録日

文字起こしする際、以下のルールに従ってください。
・カセットNo.は1-から始まる5桁のデータ（例: 1-001）
・薬品コードは6桁の半角数字で、00から始まります。
・薬品名称はカタカナは半角、漢字は全角で正確に記載。
・現在量と警告量は半角数字。
・前回登録日はYYYY/MM/DD形式。
・各列のデータは必ず,で区切ってください(csv形式で出力）。
・表のデータ以外（画像名や「文字起こししました」等の文章）は一切含めないでください。
・万が一、データが不明瞭で読み取れない場合は「不明」と記載してください。
"""

# --- 4️⃣ 画像ファイルの一覧を取得 ---
# 「cut_rows」フォルダ内で "information" を含む画像のみ対象にする
image_files = [
    f for f in os.listdir(folder_path)
    if f.lower().endswith((".jpg", ".jpeg", ".png")) and "information" in f.lower()
]


# --- 5️⃣ 結果格納用のリストを作成 ---
results = []
total_prompt_tokens = 0
total_completion_tokens = 0

# --- 6️⃣ 各画像に対してOCR実行 ---
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)

    with open(image_path, "rb") as f:
        img_bytes = f.read()
    img_base64 = base64.b64encode(img_bytes).decode("utf-8")

    response = client.chat.completions.create(
        model="gpt-4.1-mini",  # ✅ Vision対応軽量モデル
        messages=[
            {"role": "system", "content": "あなたは正確なOCR変換を行う日本語文字認識エンジンです。"},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_text},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"}}
                ]
            }
        ],
        max_tokens=2000
    )

    # --- OCR結果を取得 ---
    text_result = response.choices[0].message.content

    # --- トークン使用量を加算 ---
    usage = response.usage
    total_prompt_tokens += usage.prompt_tokens
    total_completion_tokens += usage.completion_tokens

    # --- 結果をリストに追加 ---
    results.append({
        "ファイル名": image_file,
        "文字起こし結果": text_result,
        "入力トークン": usage.prompt_tokens,
        "出力トークン": usage.completion_tokens,
        "合計トークン": usage.total_tokens
    })

# --- 7️⃣ DataFrame化して表で出力 ---
df_results = pd.DataFrame(results)
print("📋 OCR結果（各画像ごと）:")
print(df_results.to_string(index=False))

# --- 8️⃣ トークン総計とコスト計算 ---
total_tokens = total_prompt_tokens + total_completion_tokens
input_cost = total_prompt_tokens / 1000 * 0.01  # 目安：入力 0.01 USD / 1K tokens
output_cost = total_completion_tokens / 1000 * 0.03  # 目安：出力 0.03 USD / 1K tokens
total_cost = input_cost + output_cost

print("\n📊 トークン使用量（合計）:")
print(f"入力トークン（prompt）: {total_prompt_tokens}")
print(f"出力トークン（completion）: {total_completion_tokens}")
print(f"合計トークン: {total_tokens}")

print(f"\n💰 概算コスト: ${total_cost:.4f}（約¥{total_cost * 150:.2f}）")

result_dir = Path("export") / "result"
result_dir.mkdir(parents=True, exist_ok=True)

# --- CSV出力パスを指定 ---
output_csv = result_dir / "ocr_results_information.csv"

# --- CSV保存 ---
df_results.to_csv(output_csv, index=False, encoding="utf-8-sig")

print(f"\n📁 OCR結果をCSVに保存しました: {output_csv.resolve()}")


📋 OCR結果（各画像ごと）:
              ファイル名                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    文字起こし結果  入力トークン  出力トークン  合計トークン
information_1_1.jpg   1-007,002256,ニフェジピンCR錠 20mg 「トーワ」,21,20,2025/10/29  \n1-010,002381,ベニジピン塩酸塩錠4mg 「サワイ」,12,20,2025/10/29  \n1-011,000597,コントミン糖衣錠 100mg,3,20,2025/10/08  \n1-

In [4]:
# ===============================
# 📘 Step 1: GPT-4 Vision APIで表画像の文字起こし（複数画像対応・コスト算出付き）
# ===============================

from openai import OpenAI
import base64
import os
import pandas as pd

# --- 1️⃣ APIクライアントを初期化 ---
client = OpenAI()

# --- 2️⃣ OCR対象フォルダを指定 ---
folder_path = "cut_rows"

# --- 3️⃣ OCRプロンプト設定 ---
prompt_text = """
PDFもしくは画像データを文字起こしします。
文字起こしするデータは表形式で、列は以下の4列です。
薬品コード / 剤 / 薬品名称 / 使用量

文字起こしする際、以下のルールに従ってください。
・薬品コードは6桁の半角数字で、00から始まります。
・薬品名称はカタカナは半角、漢字は全角で正確に記載。
・使用量は半角数字。
・各列のデータは必ず,で区切ってください(csv形式で出力）。
・表のデータ以外（画像名や「文字起こししました」等の文章）は一切含めないでください。
・万が一読み取れない場合は「不明」と記載してください。
"""

# --- 4️⃣ 画像ファイルの一覧を取得 ---
# 「cut_rows」フォルダ内で "Usage" を含む画像のみ対象にする
image_files = [
    f for f in os.listdir(folder_path)
    if f.lower().endswith((".jpg", ".jpeg", ".png")) and "usage" in f.lower()
]


# --- 5️⃣ 結果格納用のリストを作成 ---
results = []
total_prompt_tokens = 0
total_completion_tokens = 0

# --- 6️⃣ 各画像に対してOCR実行 ---
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)

    with open(image_path, "rb") as f:
        img_bytes = f.read()
    img_base64 = base64.b64encode(img_bytes).decode("utf-8")

    response = client.chat.completions.create(
        model="gpt-4.1-mini",  # ✅ Vision対応軽量モデル
        messages=[
            {"role": "system", "content": "あなたは正確なOCR変換を行う日本語文字認識エンジンです。"},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_text},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"}}
                ]
            }
        ],
        max_tokens=2000
    )

    # --- OCR結果を取得 ---
    text_result = response.choices[0].message.content

    # --- トークン使用量を加算 ---
    usage = response.usage
    total_prompt_tokens += usage.prompt_tokens
    total_completion_tokens += usage.completion_tokens

    # --- 結果をリストに追加 ---
    results.append({
        "ファイル名": image_file,
        "文字起こし結果": text_result,
        "入力トークン": usage.prompt_tokens,
        "出力トークン": usage.completion_tokens,
        "合計トークン": usage.total_tokens
    })

# --- 7️⃣ DataFrame化して表で出力 ---
df_results = pd.DataFrame(results)
print("📋 OCR結果（各画像ごと）:")
print(df_results.to_string(index=False))

# --- 8️⃣ トークン総計とコスト計算 ---
total_tokens = total_prompt_tokens + total_completion_tokens
input_cost = total_prompt_tokens / 1000 * 0.01  # 目安：入力 0.01 USD / 1K tokens
output_cost = total_completion_tokens / 1000 * 0.03  # 目安：出力 0.03 USD / 1K tokens
total_cost = input_cost + output_cost

print("\n📊 トークン使用量（合計）:")
print(f"入力トークン（prompt）: {total_prompt_tokens}")
print(f"出力トークン（completion）: {total_completion_tokens}")
print(f"合計トークン: {total_tokens}")

print(f"\n💰 概算コスト: ${total_cost:.4f}（約¥{total_cost * 150:.2f}）")

result_dir = Path("export") / "result"
result_dir.mkdir(parents=True, exist_ok=True)

# --- CSV出力パスを指定 ---
output_csv = result_dir / "Usage.csv"

# --- CSV保存 ---
df_results.to_csv(output_csv, index=False, encoding="utf-8-sig")

print(f"\n📁 OCR結果をCSVに保存しました: {output_csv.resolve()}")


📋 OCR結果（各画像ごと）:
        ファイル名                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  文字起こし結果  入力トークン  出力トークン  合計トークン
Usage_1_1.jpg                                              002644,錠剤,カンデサルタンOD錠 8mg「サワイ」,171.5\n002291,錠剤,ドパコール配合錠L 100,1150.5\n002282,錠剤,シロスタゾールOD錠 100mg「トーワ」,77\n003201,錠剤,レベチラセタム錠 500mg「日医工」,716\n002188,錠剤,ランソプラゾールOD錠30mg「トーワ」,107\n003085,錠剤,ピタバスタチンCa・OD錠 1mg「トーワ」,2071\n003622,錠剤,ロスバスタチン錠 2.5mg「トーワ」,1040\n003288,カプセル,フルコナゾールカプセル 50mg「F」,0\n003599,錠剤,ドパコール配合錠L 50,238\n002774,錠剤,フロセミド錠 20mg「NP」,291\n002406,錠剤,(糖)ジャディアンス錠 10mg,327\n00284

In [5]:
# ============================================
# OCR文字起こしの整形 → 表（6列 or 4列）に変換＋自動補正＋半角カタカナ→全角変換
# ============================================
import pandas as pd
import re
import unicodedata  # ← 半角→全角変換に使用
from pathlib import Path

def process_ocr_csv(csv_path, label):
    # --- 1) CSVを読み込み ---
    df_raw = pd.read_csv(csv_path)

    # --- 2) ダブルクォート削除 ---
    text_series = df_raw["文字起こし結果"].astype(str).str.strip('"')

    # --- 3) 改行で展開 ---
    tmp = pd.DataFrame({
        "ファイル名": df_raw["ファイル名"],
        "文字起こし結果": text_series
    })
    tmp["行リスト"] = tmp["文字起こし結果"].str.replace("\r\n", "\n").str.replace("\r", "\n")
    tmp["行リスト"] = tmp["行リスト"].str.split("\n")

    df_lines = tmp[["ファイル名", "行リスト"]].explode("行リスト").rename(columns={"行リスト": "行テキスト"})
    df_lines = df_lines[df_lines["行テキスト"].notna() & (df_lines["行テキスト"].str.strip() != "")].copy()

    # --- 4) 補正関数（InformationとUsageで分岐）---
    def fix_line_format(line: str, label: str) -> str:
        original = line
        parts = [p.strip() for p in line.split(",")]

        # --- Information 用（カセットNoあり）---
        if label.lower() == "information":
            # カセットNo
            cassette_src = parts[0] if len(parts) >= 1 else ""
            m = re.search(r'1[-_]?(\d+)', cassette_src) or re.search(r'(\d+)', cassette_src)
            cassette_val = None
            if m:
                try:
                    cassette_val = int(m.group(1))
                except ValueError:
                    cassette_val = None
            if cassette_val is not None:
                if cassette_val > 999:
                    cassette_val = cassette_val % 1000
                parts0_fixed = f"1-{cassette_val:03d}"
            else:
                parts0_fixed = "1-000"

            # 薬品コード
            drug_src = parts[1] if len(parts) >= 2 else ""
            m2 = re.search(r'(\d+)', drug_src)
            drug_val = m2.group(1) if m2 else "000000"
            parts1_fixed = f"{int(drug_val):06d}"

            if len(parts) < 2:
                parts = [parts0_fixed, parts1_fixed]
            else:
                parts[0] = parts0_fixed
                parts[1] = parts1_fixed

        # --- Usage 用（カセットNoなし、薬品コードのみ補正）---
        elif label.lower() == "usage":
            drug_src = parts[0] if len(parts) >= 1 else ""
            m2 = re.search(r'(\d+)', drug_src)
            drug_val = m2.group(1) if m2 else "000000"
            parts0_fixed = f"{int(drug_val):06d}"
            parts[0] = parts0_fixed

        fixed = ",".join(parts)
        if fixed != original:
            print(f"🛠 修正: {original}  →  {fixed}")
        return fixed

    # --- 5) 全行に補正を適用 ---
    df_lines["行テキスト_補正後"] = df_lines["行テキスト"].apply(lambda x: fix_line_format(x, label))

    # --- 6) 半角カタカナ → 全角カタカナ変換 ---
    df_lines["行テキスト_補正後"] = df_lines["行テキスト_補正後"].apply(
        lambda x: unicodedata.normalize('NFKC', x)
    )

    # --- 7) カンマで分割 ---
    split_df = df_lines["行テキスト_補正後"].str.split(",", expand=True)

    # --- 情報源に応じて列名を切り替え ---
    if label.lower() == "information":
        expected_cols = ["カセットNo", "薬品コード", "薬品名称", "現在量", "警告量", "前回登録日"]
    elif label.lower() == "usage":
        expected_cols = ["薬品コード", "剤", "薬品名称", "使用量"]
    else:
        raise ValueError(f"Unknown label: {label}")

    # 列数が足りない場合は空列を補う
    if split_df.shape[1] < len(expected_cols):
        for _ in range(len(expected_cols) - split_df.shape[1]):
            split_df[f"欠損{_}"] = ""

    split_df = split_df.iloc[:, :len(expected_cols)]
    split_df.columns = expected_cols

    # --- 8) 前後空白除去 ---
    for c in split_df.columns:
        split_df[c] = split_df[c].astype(str).str.strip()

    # --- 9) ファイル名を付与 ---
    split_df["ファイル名"] = df_lines["ファイル名"].values

    # --- labelに応じて列順を切り替え ---
    if label.lower() == "information":
        split_df = split_df[["ファイル名", "カセットNo", "薬品コード", "薬品名称", "現在量", "警告量", "前回登録日"]]
    elif label.lower() == "usage":
        split_df = split_df[["ファイル名", "薬品コード", "剤", "薬品名称", "使用量"]]

    # --- 10) バリデーション ---
    if label.lower() == "information":
        split_df["カセットNo_OK"] = split_df["カセットNo"].str.fullmatch(r"1-\d{3}").notna()
        split_df["薬品コード_OK"]  = split_df["薬品コード"].str.fullmatch(r"00\d{4}").notna()
        split_df["現在量_OK"]     = split_df["現在量"].str.fullmatch(r"\d+").notna()
        split_df["警告量_OK"]     = split_df["警告量"].str.fullmatch(r"\d+").notna()
        split_df["前回登録日_OK"]  = split_df["前回登録日"].str.fullmatch(r"\d{4}/\d{2}/\d{2}|//").notna()
    elif label.lower() == "usage":
        split_df["薬品コード_OK"] = split_df["薬品コード"].str.fullmatch(r"00\d{4}").notna()
        split_df["使用量_OK"] = split_df["使用量"].astype(str).str.fullmatch(r"\d+").notna()

    # --- 11) 型変換 ---
    if label.lower() == "information":
        split_df["現在量"]  = pd.to_numeric(split_df["現在量"], errors="coerce").astype("Int64")
        split_df["警告量"]  = pd.to_numeric(split_df["警告量"], errors="coerce").astype("Int64")
        split_df["前回登録日_dt"] = pd.to_datetime(
            split_df["前回登録日"].replace({"//": pd.NA}),
            format="%Y/%m/%d",
            errors="coerce"
        )
    elif label.lower() == "usage":
        split_df["使用量"] = (
            pd.to_numeric(split_df["使用量"], errors="coerce")
            .apply(lambda x: int(x) if pd.notna(x) else pd.NA)
            .astype("Int64")
        )

    # --- 12) 出力（整形確認） ---
    print("📋 整形後の表データ（先頭5行）:")
    print(split_df.head().to_string(index=False))

    # --- 13) 保存 ---
    # 既に上流で today_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") が定義されている前提
    output_dir = Path("export") / today_str
    output_dir.mkdir(parents=True, exist_ok=True)

    out_csv = output_dir / f"{label}.csv"
    split_df.to_csv(out_csv, index=False, encoding="utf-8-sig")

    print(f"\n📁 保存完了: {out_csv.resolve()}")


# --- 関数呼び出し部分 ---
process_ocr_csv(r"export/result/ocr_results_information.csv", "Information")
process_ocr_csv(r"export/result/Usage.csv", "Usage")


🛠 修正: 1-007,002256,ニフェジピンCR錠 20mg 「トーワ」,21,20,2025/10/29    →  1-007,002256,ニフェジピンCR錠 20mg 「トーワ」,21,20,2025/10/29
🛠 修正: 1-010,002381,ベニジピン塩酸塩錠4mg 「サワイ」,12,20,2025/10/29    →  1-010,002381,ベニジピン塩酸塩錠4mg 「サワイ」,12,20,2025/10/29
🛠 修正: 1-011,000597,コントミン糖衣錠 100mg,3,20,2025/10/08    →  1-011,000597,コントミン糖衣錠 100mg,3,20,2025/10/08
🛠 修正: 1-012,000669,ジゴシン錠 0.125mg,42,20,2025/09/02    →  1-012,000669,ジゴシン錠 0.125mg,42,20,2025/09/02
🛠 修正: 1-013,003029,セレコキシブ 錠 100mg 「ニプロ」,46,30,2025/10/04    →  1-013,003029,セレコキシブ 錠 100mg 「ニプロ」,46,30,2025/10/04
🛠 修正: 1-014,003056,ハロペリドール錠 3mg 「アメル」,38,20,2025/06/11    →  1-014,003056,ハロペリドール錠 3mg 「アメル」,38,20,2025/06/11
🛠 修正: 1-015,002432,クエチアピン錠 100mg 「アメル」,148,20,2025/10/02    →  1-015,002432,クエチアピン錠 100mg 「アメル」,148,20,2025/10/02
🛠 修正: 1-016,000920,チラージンS錠 50μg,24,30,2025/10/02    →  1-016,000920,チラージンS錠 50μg,24,30,2025/10/02
🛠 修正: 1-017,000326,エビリファイ錠 12mg,14,0,2025/08/19    →  1-017,000326,エビリファイ錠 12mg,14,0,2025/08/19
🛠 修正: 1-018,002440,ベラプロストNa錠 20μg 「NIG」,55,0

In [ ]:
# ============================================
# 💊 Information.csv と Usage.csv のマージ処理
#   - カセット番号ありの薬品のみ対象
#   - 使用量を月平均換算
#   - 差分・補充数を計算
#   - HTMLレポートとして出力
# ============================================
import pandas as pd
import math
from pathlib import Path

# --- 基本設定 ---
month_count = 2  # 使用量を2か月分→1か月平均に換算
output_dir = Path("export") / today_str  # 例: export/2025-11-03_16-45-22
html_path = output_dir / "マージレポート.html"

# --- 1️⃣ CSV読み込み ---
info_df = pd.read_csv(output_dir / "Information.csv", dtype=str)
usage_df = pd.read_csv(output_dir / "Usage.csv", dtype=str)

# --- 2️⃣ 必要な列のみ抽出 ---
info_df = info_df[["カセットNo", "薬品コード", "薬品名称", "現在量", "前回登録日"]]
usage_df = usage_df[["薬品コード", "剤", "薬品名称", "使用量"]]

# --- 3️⃣ 数値列をfloat変換 ---
info_df["現在量"] = pd.to_numeric(info_df["現在量"], errors="coerce")
usage_df["使用量"] = pd.to_numeric(usage_df["使用量"], errors="coerce")

# --- 4️⃣ カセット番号ありのみ抽出 ---
info_df = info_df[info_df["カセットNo"].notna() & (info_df["カセットNo"].str.strip() != "")]

# --- 5️⃣ UsageとInformationを薬品コードでマージ ---
merged = pd.merge(info_df, usage_df, on="薬品コード", how="inner", suffixes=("_info", "_usage"))

# --- 6️⃣ 差分と補充数を計算 ---
def calc_replenish(row):
    usage_per_month = math.floor(row["使用量"] / month_count) if pd.notna(row["使用量"]) else 0
    diff = usage_per_month - (row["現在量"] if pd.notna(row["現在量"]) else 0)

    # 初期設定
    replenish = 0
    note = ""

    if diff < 30:
        replenish = 0
    elif diff < 60:
        replenish = 50
    elif diff < 110:
        replenish = 100
    elif diff < 160:
        replenish = 150
    elif diff < 210:
        replenish = 200
    elif diff < 310:
        replenish = 300
    else:
        step = math.floor((diff - 310) / 100) + 1
        replenish = 300 + (step * 100)

    # 個別ルール
    upperLimit100 = ["002402"]
    multiple21 = ["001227"]
    multiple14 = ["003035","003084"]
    skipCalc = ["000113", "001649","002913","001310","002510","001605","002747","002578","000004","000047","002374","000985","001146","002330","001367","001644","001751","002263","000477","002187","003622","003085","003201","002291"]

    if row["薬品コード"] in upperLimit100:
        if replenish > 100:
            replenish = 100
        note = "上限100錠"
    elif row["薬品コード"] in multiple21:
        unit = 21
        replenish = math.ceil(diff / unit) * unit
        note = "21錠シート"
    elif row["薬品コード"] in multiple14:
        unit = 14
        replenish = math.ceil(diff / unit) * unit
        note = "14錠シート"
    elif row["薬品コード"] in skipCalc:
        replenish = "B"
        note = "バラ錠あり"

    return pd.Series([usage_per_month, diff, replenish, note],
                     index=["使用量（月平均）", "差分", "補充数", "備考"])

calc_df = merged.apply(calc_replenish, axis=1)
merged = pd.concat([merged, calc_df], axis=1)

# 「1-XXX」から数字部分（XXX）を取り出し、整数化してソートキーにする
merged["カセット番号_sort"] = (
    merged["カセットNo"]
    .astype(str)
    .str.replace("1-", "", regex=False)  # "1-" を削除
    .astype(int)                          # 数値化
)
# 数値昇順でソート
merged = merged.sort_values(by="カセット番号_sort", ascending=True).reset_index(drop=True)


# --- 7️⃣ HTML生成（完全改良版）---
style = """
<style>
body {
  font-family: 'Meiryo', sans-serif;
  font-size: 7pt;
  margin: 12mm;
}

/* ===== テーブル全体 ===== */
table {
  border-collapse: collapse;
  width: 100%;
  table-layout: fixed;
  border: 1px solid #999;
}

th, td {
  border: 1px solid #999;
  padding: 2px 3px;
  line-height: 1.05;
  word-wrap: break-word;
}

/* ===== 行の高さ調整 ===== */
td {
  min-height: 2.4em;  /* 2行分くらいの高さを確保 */
  height: 2.4em;      /* 固定高さとしても適用（印刷で安定） */
}

/* ===== ヘッダー ===== */
th {
  background: #f3f3f3;
  font-weight: bold;
  text-align: center;
}

/* ===== 各列の幅指定 ===== */
/* ※ 合計100%を維持するようバランス調整 */
th:nth-child(1), td:nth-child(1) { width: 70px; text-align: center; }   /* 薬品コード */
th:nth-child(2), td:nth-child(2) { width: 60px; text-align: center; }   /* カセットNo */
th:nth-child(3), td:nth-child(3) { width: 26%; text-align: left; }      /* 薬品名称（広め） */
th:nth-child(4), td:nth-child(4) { width: 55px; text-align: center; }   /* 差分 */
th:nth-child(5), td:nth-child(5) { width: 55px; text-align: center; }   /* 補充数 */
th:nth-child(6), td:nth-child(6) { width: 13%; text-align: left; }      /* 備考（やや広め） */
th:nth-child(7), td:nth-child(7) { width: 55px; text-align: center; }   /* 使用量（月平均） */
th:nth-child(8), td:nth-child(8) { width: 55px; text-align: center; }   /* 現在量 */
th:nth-child(9), td:nth-child(9) { width: 100px; text-align: center; }   /* 最終更新日 */

/* === 重要エリア（カセットNo〜備考）を淡いグレーで強調 === */
td:nth-child(2),
td:nth-child(3),
td:nth-child(4),
td:nth-child(5),
td:nth-child(6) {
  background-color: #f7f7f7 !important; /* ごく淡いグレー (#f5f5f5 〜 #f7f7f7 が自然) */
  font-weight: 500;                     /* 少しだけ太字にして視線誘導 */
  -webkit-print-color-adjust: exact;
  print-color-adjust: exact;
}

/* 境界線をやや強調してブロック感を出す */
td:nth-child(2) {
  border-left: 2px solid #aaa;  /* 左端を濃いグレーで引き締める */
}
td:nth-child(6) {
  border-right: 2px solid #aaa; /* 備考の右側も締める */
}

/* 補充行（黄色）との重なりを自然にするため上書き設定 */
tr.highlight td:nth-child(2),
tr.highlight td:nth-child(3),
tr.highlight td:nth-child(4),
tr.highlight td:nth-child(5),
tr.highlight td:nth-child(6) {
  background-color: #fff7ce !important; /* 補充対象行は黄色を優先 */
  font-weight: bold;
}



/* ===== 行スタイル ===== */
tr.highlight td { background-color: #fff7ce !important; }  /* 黄色強調: 印刷でも残る */
tr.blue td { color: #0044cc; }
tr.noneed td { opacity: 0.6; }

/* ===== 補充数セルの特別装飾 ===== */
.replenish-highlight {
  color: red;
  font-weight: bold;
}

/* ===== 太字強調（補充行の特定列） ===== */
.bold-col {
  font-weight: bold;
}

/* ===== 印刷対応設定 ===== */
@media print {
  body { margin: 5mm; }
  th { 
    background: #f3f3f3 !important;
    -webkit-print-color-adjust: exact;
    print-color-adjust: exact;
  }
  tr.highlight td {
    background-color: #fff7ce !important;
    -webkit-print-color-adjust: exact;
    print-color-adjust: exact;
  }
  thead { display: table-header-group; }  /* ページ繰り返し */
}
</style>
"""

# --- 行ごとのクラス判定 --- #
def row_class(row):
    if row["補充数"] == 0:
        return "noneed"
    elif row["補充数"] == "B":
        return "blue"
    else:
        return "highlight"

merged["row_class"] = merged.apply(row_class, axis=1)

# --- HTMLテーブルの作成（列の並びを指定） --- #
html_table = """
<table>
<thead>
<tr>
  <th>薬品コード</th>
  <th>カセットNo</th>
  <th>薬品名称</th>
  <th>差分</th>
  <th>補充数</th>
  <th>備考</th>
  <th>使用量（月平均）※</th>
  <th>現在量</th>
  <th>最終更新日</th>
</tr>
</thead>
<tbody>
"""

for _, r in merged.iterrows():
    # --- 補充数セル：赤太字 ---
    if isinstance(r["補充数"], (int, float)) and r["補充数"] > 0:
        replenish_html = f"<span class='replenish-highlight'>{int(r['補充数'])}</span>"
    else:
        replenish_html = r["補充数"]

    # --- 補充がある行の場合、カセットNo・薬品名称・差分を太字に ---
    bold_start = "<span class='bold-col'>" if r["row_class"] == "highlight" else ""
    bold_end = "</span>" if r["row_class"] == "highlight" else ""

    html_table += f"""
    <tr class='{r["row_class"]}'>
        <td>{r["薬品コード"]}</td>
        <td>{bold_start}{r["カセットNo"]}{bold_end}</td>
        <td>{bold_start}{r["薬品名称_info"]}{bold_end}</td>
        <td>{bold_start}{r["差分"]}{bold_end}</td>
        <td>{replenish_html}</td>
        <td>{r["備考"]}</td>
        <td>{r["使用量（月平均）"]}</td>
        <td>{r["現在量"]}</td>
        <td>{r["前回登録日"]}</td>
    </tr>
"""
html_table += "</tbody></table>"

# --- HTML出力 --- #
html_output = f"""
<html><head><meta charset='UTF-8'>{style}</head><body>
<h2>錠剤分包機　カセット補充数集計表</h2>
{html_table}
<p style='color:#555;text-align:right;font-size:8pt;margin-top:8px;'>
※ 使用量（月平均）は {month_count}ヶ月分の使用量を集計し、1ヶ月あたりに換算しています。<br>
※ 黄色行は補充対象、青文字はバラ錠扱いです。
</p>
</body></html>
"""

# --- 保存 --- #
html_path.write_text(html_output, encoding="utf-8-sig")
print(f"✅ マージレポートを作成しました: {html_path.resolve()}")


✅ マージレポートを作成しました: C:\Users\PC_User\Desktop\アプリ作成\OCR-DrugTableConverter\export\2025-11-06_10-51-26\マージレポート.html
